In [ ]:
import numpy as np
from util import read_alu
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float
from numpy import linalg as LA
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
# dp = al.Depot()
# runner = dp.Runner()
# v = dp.create_coated((num_samples), 3)
# s = dp.create_coated((num_samples), 1)
num_samples = 30 * 30 * 30

In [ ]:
truth_dir = '/media/kennychufk/mldata/alluvion-data'
dirnames = [f"{truth_dir}/diagonal-train4/rltruth-dbdf3f05-0914.01.20.58",
            f"{truth_dir}/diagonal-train4/rltruth-8cecae4f-0914.01.50.28",
            f"{truth_dir}/diagonal-train4/rltruth-d949a937-0914.02.21.44",
            f"{truth_dir}/diagonal-train4/rltruth-9d4aaf45-0914.02.52.46",
            f"{truth_dir}/diagonal-train4/rltruth-448949d6-0914.03.23.47",
            f"{truth_dir}/diagonal-train4/rltruth-33cbd54f-0914.03.59.31",
            f"{truth_dir}/diagonal-train4/rltruth-d4537532-0914.04.30.48",
            f"{truth_dir}/diagonal-train4/rltruth-350aefb5-0914.05.02.10",
            f"{truth_dir}/diagonal-train4/rltruth-22ba32a5-0914.05.33.45",
            f"{truth_dir}/diagonal-train4/rltruth-1dc784ae-0914.06.08.59",
            f"{truth_dir}/diagonal-train4/rltruth-73469bdd-0914.06.43.30",
            f"{truth_dir}/diagonal-train4/rltruth-8dfe9db8-0914.07.17.17",
            f"{truth_dir}/diagonal-train4/rltruth-a35ca720-0914.07.49.48",
            f"{truth_dir}/diagonal-train4/rltruth-ebc119cb-0914.08.22.41",
            f"{truth_dir}/diagonal-train4/rltruth-3c127f13-0914.08.55.27",
            f"{truth_dir}/diagonal-train4/rltruth-d24a33dc-0914.09.29.59",
            f"{truth_dir}/diagonal-train4/rltruth-cada1ed3-0914.10.04.19",
            f"{truth_dir}/diagonal-train4/rltruth-87b12884-0914.10.41.05",
            f"{truth_dir}/diagonal-train4/rltruth-312c0cc8-0914.11.53.13",
            f"{truth_dir}/diagonal-train4/rltruth-a93a68aa-0914.12.26.32",
            f"{truth_dir}/diagonal-train4/rltruth-b9b86c25-0914.12.59.04",
            f"{truth_dir}/diagonal-train4/rltruth-2d9afd64-0914.13.35.00",
            f"{truth_dir}/diagonal-train4/rltruth-b5a1fc48-0914.14.09.07",
            f"{truth_dir}/diagonal-train4/rltruth-2a98f2fb-0914.14.42.39",
            f"{truth_dir}/diagonal-train4/rltruth-cf367ec6-0914.15.16.06",
            f"{truth_dir}/diagonal-train4/rltruth-12ff5012-0914.15.52.21",
            f"{truth_dir}/diagonal-train4/rltruth-b79afb3c-0914.16.27.42",
            f"{truth_dir}/diagonal-train4/rltruth-2a7d4859-0914.17.10.05",
            f"{truth_dir}/diagonal-train4/rltruth-5d9f1cd4-0914.17.47.12",
            f"{truth_dir}/diagonal-train4/rltruth-882bd11e-0914.18.20.53",
            f"{truth_dir}/diagonal-train4/rltruth-8150d9a2-0914.18.58.36",
            f"{truth_dir}/diagonal-train4/rltruth-8834d9d4-0914.19.38.36",
            f"{truth_dir}/diagonal-train4/rltruth-5640dc4e-0914.20.17.18",
            f"{truth_dir}/diagonal-train4/rltruth-b9d5ee38-0914.20.51.11",
            f"{truth_dir}/diagonal-train4/rltruth-b5351c5b-0914.21.28.35",
            f"{truth_dir}/diagonal-train4/rltruth-aff784dd-0914.22.04.00",
            f"{truth_dir}/diagonal-train4/rltruth-9196b61d-0914.22.41.50",
            f"{truth_dir}/diagonal-train4/rltruth-9c9a292c-0914.23.16.52",
            f"{truth_dir}/diagonal-train4/rltruth-1654bb10-0914.23.56.12",
            f"{truth_dir}/diagonal-train4/rltruth-65288f7b-0915.00.38.11",
            f"{truth_dir}/diagonal-train4/rltruth-f01c96fb-0915.01.15.43",
            f"{truth_dir}/diagonal-train4/rltruth-2d482973-0915.01.58.26",
            f"{truth_dir}/diagonal-train4/rltruth-d94bc10d-0915.02.33.42",
            f"{truth_dir}/diagonal-train4/rltruth-e2f978b2-0915.03.10.03",
            f"{truth_dir}/diagonal-train4/rltruth-1af3c395-0915.04.02.08",
            f"{truth_dir}/diagonal-train4/rltruth-44eba036-0915.04.44.08",
            f"{truth_dir}/diagonal-train4/rltruth-227ecffa-0915.05.25.49",
            f"{truth_dir}/diagonal-train4/rltruth-ee7af950-0915.06.02.41",
            f"{truth_dir}/diagonal-train4/rltruth-14f08550-0915.06.44.03"]

scene_speed_collections = []
buoy_counts = []
for dirname in dirnames:
    print(np.load(f'{dirname}/num_buoys.npy'))
    buoy_counts.append(np.load(f'{dirname}/num_buoys.npy').item())
    frame_speeds = []
    total_num_speeds = 0
    for frame_id in range(1000):
        s = LA.norm(read_alu(f'{dirname}/v-{frame_id}.alu'), axis=1)
        weight = read_alu(f'{dirname}/density-weight-{frame_id}.alu')
        masked = s[weight.ravel()==1]
        masked_non_zero = masked[masked>0]
        frame_speeds.append(masked_non_zero)
        total_num_speeds+=len(masked_non_zero)

    scene_speeds = np.empty(total_num_speeds, np.float32)
    cursor = 0
    for frame_speed in frame_speeds:
        frame_size = len(frame_speed)
        scene_speeds[cursor:cursor+frame_size] = frame_speed
        cursor+=frame_size
    scene_speed_collections.append(scene_speeds)

In [ ]:
buoy_counts = np.array(buoy_counts)
percentile10_trends = np.zeros(len(dirnames))
percentile25_trends = np.zeros(len(dirnames))
percentile50_trends = np.zeros(len(dirnames))
percentile75_trends = np.zeros(len(dirnames))
percentile90_trends = np.zeros(len(dirnames))
for i, dirname in enumerate(dirnames):
    percentile10_trends[i] = np.percentile(scene_speed_collections[i], 10)
    percentile25_trends[i] = np.percentile(scene_speed_collections[i], 25)
    percentile50_trends[i] = np.percentile(scene_speed_collections[i], 50)
    percentile75_trends[i] = np.percentile(scene_speed_collections[i], 75)
    percentile90_trends[i] = np.percentile(scene_speed_collections[i], 90)

In [ ]:
populate_plt_settings(plt)

In [ ]:
reg10 = LinearRegression().fit(buoy_counts.reshape(-1, 1), percentile10_trends)
reg25 = LinearRegression().fit(buoy_counts.reshape(-1, 1), percentile25_trends)
reg50 = LinearRegression().fit(buoy_counts.reshape(-1, 1), percentile50_trends)
reg75 = LinearRegression().fit(buoy_counts.reshape(-1, 1), percentile75_trends)
reg90 = LinearRegression().fit(buoy_counts.reshape(-1, 1), percentile90_trends)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=get_fig_size(get_column_width()))
ax.plot(buoy_counts, percentile90_trends, label='90th percentile')
ax.plot(buoy_counts, percentile75_trends, label='75th percentile')
ax.plot(buoy_counts, percentile50_trends, label='50th percentile')
ax.plot(buoy_counts, percentile25_trends, label='25th percentile')
ax.plot(buoy_counts, percentile10_trends, label='10th percentile')




ax.set_ylabel(r"Speed ($\textup{m}\textup{s}^{-1}$)")
ax.set_xlabel("No. of buoys")
ax.set_xlim(0,100)
ax.legend(facecolor='white', frameon=True, edgecolor='None')
fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633
# fig.savefig('buoy-obstruction.pgf')

In [ ]:
reg10.coef_[0]/reg10.intercept_*100

In [ ]:
reg25.coef_[0]/reg25.intercept_*100

In [ ]:
reg50.coef_[0]/reg50.intercept_*100

In [ ]:
reg75.coef_[0]/reg75.intercept_*100

In [ ]:
reg90.coef_[0]/reg90.intercept_*100